# Genetic Markowitz

The goal of this notebook is to explore the optimal mean-variance portfolio and to formulate the efficient frontier in multiobjective optimization terms.

The optimal mean-variance portfolio, as definded by Markowitz, is a portfolio that maximizes returns and minimizes risk, understood as the variance of the portfolio. However the weight of each of those objectives depends on the risk aversion of the individual.
As such, the problem becomes
$$ \min_{w} \quad \frac{1}{2} w' \Sigma w - \lambda R' w $$
$$ \textrm{s.t.} \quad e'w=1$$

Solving and iterating through different values of $ \lambda $ you get the efficient frontier, the return-variance mix of the different portfolios for different risk aversion profiles.
Since the problem is a convex, quadratic optimization problem it is quite straightforward to solve numerically.

In [1]:
import numpy as np
import plotly.graph_objects as go
from scipy.optimize import minimize

In [23]:
MU = np.array([.05,.07,.03])
COV = np.array([[.004,.002,-.001],[.002,.012,.001],[-.001,.001,.02]])
X0 = np.array([1/3,1/3,1/3])
N = 100

def obj_function(w: np.ndarray,l=1,R=MU,cov=COV):
    w = w/np.sum(w)
    return w.T@cov@w/2 - l*R.T@w

def calc_std(w,cov=COV):
    return np.sqrt(w.T@cov@w)

def calc_rets(w,R=MU):
    return R.T@w


In [24]:
lambda_vals = np.linspace(0,.8,N)
stds = np.zeros(N)
rets = np.zeros(N)
for i,l in enumerate(lambda_vals):
    sol = minimize(obj_function,X0,args=(l,MU,COV))
    w = np.array(sol.x)
    stds[i] = calc_std(w)
    rets[i] = calc_rets(w)

In [25]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=stds,y=rets,mode='lines',name='quadratic'))
fig.add_trace(go.Scatter(x=np.sqrt(COV.diagonal()),y=MU,mode="markers",name='assets'))
fig.
fig.show()

The problem is completely unconstrained regarding the weights, apart from having the sum equal 1. This means that shorting the assets is allowed, which explains how it is possible to have portfolios riskier than the riskiest portfolio (which are obtained by building a portfolio with a weight higher than 1 for that asset and shorting the other assets).

Now, lets go with the multiobjective formulation.
Credits go to https://pymoo.org/case_studies/portfolio_allocation.html

In [31]:
from pymoo.core.problem import ElementwiseProblem
from pymoo.core.repair import Repair
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.optimize import minimize

In [32]:
class PortfolioProblem(ElementwiseProblem):

    def __init__(self, mu, cov, **kwargs):
        super().__init__(n_var=len(mu), n_obj=2, xl=0.0, xu=1.0, **kwargs)
        self.mu = mu
        self.cov = cov

    def _evaluate(self, x, out, *args, **kwargs):
        exp_return = x @ self.mu
        exp_risk = np.sqrt(x.T @ self.cov @ x)
        out["F"] = [exp_risk, -exp_return]

class PortfolioRepair(Repair):

    def _do(self, problem, X, **kwargs):
        X[X < 1e-3] = 0
        return X / X.sum(axis=1, keepdims=True)

In [33]:
problem = PortfolioProblem(MU, COV)

algorithm = NSGA2(repair=PortfolioRepair())

res = minimize(problem,
               algorithm,
               seed=42,
               verbose=True)
X, F = res.opt.get("X", "F")

n_gen  |  n_eval  | n_nds  |      eps      |   indicator  
     1 |      100 |     18 |             - |             -
     2 |      200 |     34 |  0.1606679219 |         ideal
     3 |      300 |     49 |  0.0090864653 |             f
     4 |      400 |     70 |  0.0082425421 |             f
     5 |      500 |    100 |  0.0302558901 |         ideal
     6 |      600 |    100 |  0.0355483668 |         ideal
     7 |      700 |    100 |  0.0450369046 |         ideal
     8 |      800 |    100 |  0.0024575457 |             f
     9 |      900 |    100 |  0.0128205974 |         ideal
    10 |     1000 |    100 |  0.0038539440 |         nadir
    11 |     1100 |    100 |  0.0022151389 |             f
    12 |     1200 |    100 |  0.0064715892 |         ideal
    13 |     1300 |    100 |  0.0035812792 |         nadir
    14 |     1400 |    100 |  0.0014375787 |             f
    15 |     1500 |    100 |  0.0025325303 |             f
    16 |     1600 |    100 |  0.0067973499 |         nad

In [34]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=F[:,0],y=-F[:,1],mode='markers',name='genetic'))
fig.add_trace(go.Scatter(x=stds,y=rets,mode='lines',name='quadratic'))
fig.add_trace(go.Scatter(x=np.sqrt(COV.diagonal()),y=MU,mode="markers",name='assets'))
fig.show()

As we can see here, the solutions are practically identical. There is however a slight difference towards the risky side of the portfolios. For the genetic solution, the riskiest possible portfolio is holding just the riskiest asset. That is because we have constrained the second problem so that no shorting is allowed, so it is impossible to hold more than 1 of any asset.

For this portfolio optimization problem, as you have seen it makes no sense to use genetic or any other type of optimization as the quadratic formulation is easy enough.
However, this exercise was meant to showcase how the canonical formulation of the portfolio optimization problem was just an optimization problem trying to maximize an objective and minimize another. Variance was chosen as the risk measure to minimize, and that allows for very elegant mathematical solutions to the problem. However, other risk measures and other non-quadratic or even non-convex portfolio optimization formulations are possible, and with the evolution of optimization technologies it is becoming easier and easier to solve these alternative formulations.